<img src="images/Frame_108.png" style="margin:auto"/>

___

# Interactive Nena-API Tutorial With Python

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple
plt.style.use('ggplot')

## Introduction

StormGeo Nena analysis currently offers a REST-API for fetching most of the data available on the [fundamentals](https://nena.no/secure/fundamentals/) page of Nena analysis.
In this interactive tutorial we will: 
* Show how to use the Python Requests package for for fetching data from the API
* Use the Pandas library to read-in and plot the data from the API.

Our API has three endpoints:
* */api/fundamental/meta* - Metadata for a single data series.
* */api/fundamental/meta/all* - Metadata for a group of series with identical SeriesId prefix.
* */api/fundamental/series* - Series data. I.e. the same data shown on the fundamentals page.
___

## Accessing to the API

User authentication is done by generating an access token. The token will be valid for 15 minutes, after which you
need to request a new token. The access token needs to be submitted in all requests.
We will now go through how to generate an access token using your Nena user login credentials. To proceed with the tutorial assign your username and password to __USER__ and __PASS__.

In [ ]:
ROOT = "https://api.nena.no"  # Root url
USER = ""#""  # Username
PASS = ""  # Password

### Obtain an access token

To obtain an access token we must submit a post request to the */api/user/login/* endpoint. The request must contain
a json body with your username and password. Here we have written a simple function called *obtain_access_token()*, which takes your user login and returns an access token. 

In [ ]:
def obtain_access_token(username: str, password: str, root_url: str) -> Tuple:
    """ Function for obtaining an API access token.
    Args
    username [str] : nena account username
    password [str] : nena account password
    root_url [str] : api root url.
    Returns
    (username, token) [tuple] : A tuple containing username and access token.
    """
    endpoint_auth = "/api/user/login"

    userdata = {
      "UserName": f"{username}",
      "Password": f"{password}"
    }
    url = root_url + endpoint_auth
    res_auth = requests.post(url=url, json=userdata).json()
    return (res_auth['UserName'], res_auth['Token'], root_url)

In [ ]:
username, token, _ = obtain_access_token(USER, PASS, ROOT)
print(f'Your acccess token: {token}')

___

## Retrieve metadata on a single series

With our access token we can retrieve data from the API. We will start by fetching metadata for a single series.

### Where do I find the Series ID?

You can find any series id by goint to the [fundamentals](https://nena.no/secure/fundamentals/) page, selecting a series and noting the id in the metadata table. <br>Let's find the series containing "Hourly Wind Production" for UK. <br>

We see that the Series ID for "Hourly Wind Production in the UK" is called: __"ukhrwind"__.

![](images/finding_series_id_on_fundamentals_page.PNG)

With our Series ID we can now retrieve addition metadata by using the function below:

In [ ]:
def retrieve_series_metadata(series_id: str, user_auth: Tuple):
    """ Function which retrieves single series metadata.
    Args
    series_id [str] : Id for wanted series.
    user_auth [Tuple] : Tuple with username and access token.
    Returns
    res_single_series [Dict] : Dictionary with series meta data.
    """
    username, token, root_url = user_auth
    endpoint_meta_single = "/api/fundamental/meta"
    url = root_url + endpoint_meta_single

    form_input = {
      "SeriesId": f"{series_id}",
      "UserInfo": {
        "UserName": f"{username}",
        "Token": f"{token}"
      }
    }
    res_single_series = requests.post(url=url, json=form_input)
    return res_single_series.json()

In [ ]:
res_metadata_single_series = retrieve_series_metadata(
    series_id="ukhrwind",
    user_auth=obtain_access_token(USER, PASS, ROOT)
)

df_single = pd.DataFrame.from_dict(
    res_metadata_single_series,
    orient='index',
    columns=['Value'])

df_single

___

## Retrieve series metadata by performing prefix search 

Let's now try to find series metadata for many series. This is usefull when you want to fetch data from series which are related, for example by country or model. Our goal now is to find all the renewable energy production data and spot price for the UK. <br>

We use the __/api/fundamental/meta/all__ endpoint to find every Series ID related by a common prefix. We can use this method to find series matching a
country or forecast model:

* Prefix: "uk" &rarr; Gets all series meta data related to UK.
* Prefix: "nhse" &rarr; Gets all series related to the nordic balance for Sweden.


### How can I find a prefix?

With prefix we simple mean the first few letters of a series id. For example, all data series related to UK 
have a series id which start with "uk". You can always go to the fundamentals page and find usefull prefix search keywords. As shown below we can
find all Nordic Balance series related to Sweden by looking at the following prefix.

![](images/finding_prefix_on_fundamentals_page.PNG)

In [ ]:
def retrieve_metadata_by_prefix(prefix: str, user_auth: Tuple):
    """ Function which retrieves metadata for many data series with common
    series id prefix.
    
    Ex. We can search by country using a prefix:
        - "uk"
        - "ukhr"
        - "ukday"
    or we can search by forecast model:
        - "nh" : Series found in the nordic balance.
        - "nhse" : Series related to Swedish nordic balance.
    Args
    prefix [str] : The prefix search keyword. These can be found on Nena
    fundamentals.
    user_auth [Tuple] : Tuple with username and access token.
    Returns
    (res_all_serie, series_ids) [tuple] : Tuple with request result and a list of
    all seriesIds.
    """
    endpoint_allseries = "/api/fundamental/meta/all"
    username, token, root_url = user_auth
    
    url = root_url + endpoint_allseries
    
    form_all = {
      "Prefix": f"{prefix}",
      "UserInfo": {
        "UserName": f"{username}",
        "Token": f"{token}"
      }
    }

    res_all_series = requests.post(url=url, json=form_all).json()

    series_ids = []
    for series in res_all_series:
        series_ids.append(series['Code'])
    return (res_all_series, series_ids)

In [ ]:
meta_data_all, series_codes = retrieve_metadata_by_prefix(
    prefix="uk",
    user_auth=obtain_access_token(USER, PASS, ROOT),
)
print('All Series IDs related to the UK \n')
print(series_codes)

___

## Retrieving series data

Once we have the series ids we can start to fetch the actual series data. We will start by writing a function witch fetches a single data series. This function will be re-used to fetch many series later.

In [ ]:
def get_series_data(
    series_id: str, user_auth: Tuple, from_date, to_date, resolution: str
) -> Dict:
    """Function to get data for a series.
    Args
    series_id [str] : Series ID
    user_auth [Tuple] : Tuple with username and access token.
    from_date [str] : datetime string on the format 2022-01-10
    to_date [str] : datetime string on the format 2022-01-20
    resolution [str] : keyword to filter and re-sample data. See the documentation.
    Returns
    series_result [Dict] : Dictionary of request result.
    """
    username, token, root_url = user_auth
    endpoint_series_data = "/api/fundamental/series"
    url = root_url + endpoint_series_data

    series_data = {
        "FromDateTime": f"{from_date}",
        "ToDateTime": f"{to_date}",
        "Resolution": f"{resolution}",
        "SeriesId": f"{series_id}",
        "UserInfo": {"UserName": f"{username}", "Token": f"{token}"},
    }

    series_result = requests.post(url=url, json=series_data).json()
    return series_result

The date information needs to be on a valid datetime string format. Ex: "2021-12-01", but it is also possible to use keywords like "today+30". The *Resolution* parameter refers to the time resolution of the time series data. Here we can specify either "hr", "day", or "week" to resample the time series to the corresponding time interval.

In [ ]:
uk_price_series = 'ukhrprice'

uk_series_result_hr = get_series_data(
    series_id=uk_price_series,
    user_auth=obtain_access_token(USER, PASS, ROOT),
    from_date='today-30', ## We go back 30 days in time
    to_date='today+14', ## We want the 14 days forecasted data as well.
    resolution='hr'
)

We use Pandas to convert the returned json data from our request into a dataframe. We can then plot the data using f.ex. pandas built-in plotting methods.

In [ ]:
df = pd.DataFrame.from_dict(
    uk_series_result_hr['Values'],
    orient='index',
    columns=[f'{uk_price_series}']
)

df.index = pd.to_datetime(df.index)

fig, ax = plt.subplots()
df.plot(
    rot=10,
    figsize=[10, 5],
    ylabel=f"{uk_series_result_hr['Meta']['description']}",
    xlabel="Time",
    ax=ax
)

fig.tight_layout()

___

## Retrieving several data series

Let's now try to fetch several data series using the list of series ids (__series_codes__) we got from the prefix metadata search. Here we use the previous *get_series_data()* function and loop over the series ids.

In [ ]:
user_auth = obtain_access_token(USER, PASS, ROOT)  # We generate an access token before the loop.

df_list = []
for series in series_codes:
    json_data = get_series_data(
        series_id=series,
        user_auth=user_auth,
        from_date='today-30',
        to_date='today+14',
        resolution='hr'
    )
    df_temp = pd.DataFrame.from_dict(
        json_data['Values'],
        orient='index',
        columns=[f"{series}"]
    )
    df_list.append(df_temp)
df_tot = pd.concat(df_list, axis=1)  # Concat all the dataframes
df_tot.index = pd.to_datetime(df_tot.index)  # Set the index to a time series

Now we can plot the production of renewable energy and the UK spot price.

In [ ]:
fig_tot, ax_tot = plt.subplots(
    nrows=2, gridspec_kw={'height_ratios': [1.5, 3]})

df_tot[['ukhrprice']].plot(
    rot=10,
    figsize=[12, 6],
    ylabel="Eur/MWh",
    ax=ax_tot[0],
    color='black'
)

ax_tot[0].xaxis.set_visible(False)


df_tot[['ukhrwind', 'ukhrsolar']].plot(
    rot=10,
    figsize=[12, 6],
    ylabel="GWh/h",
    xlabel='Time / month-day',
    ax=ax_tot[1]
)

fig_tot.tight_layout()
fig_tot.subplots_adjust(hspace=0.05)

This concludes the Nena API interactive tutorial. If you have any further questions or want an in-person overview of the API, please don't hesitate to contact us at StormGeo Energy Markets